In [23]:
%pip install mediapipe opencv-python matplotlib   #installing the required libraries

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#installing'

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np

### Realtime Camera Feed

In [35]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read() #ret: A boolean indicating whether the frame was successfully captured.
                            #frame: The actual image frame from the video feed.
    
    cv2.imshow('Selfie Seg', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### For Black or Solid Background


In [42]:

# Initialize MediaPipe Selfie Segmentation
mp_selfie = mp.solutions.selfie_segmentation

# Start video capture
cap = cv2.VideoCapture(0)

# Use 'with' statement for the segmentation model
with mp_selfie.SelfieSegmentation(model_selection=1) as model:  # Model 1 = Higher quality segmentation
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB (MediaPipe requires RGB input)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with segmentation model
        res = model.process(frame_rgb)

        # Extract the segmentation mask
        mask = res.segmentation_mask

        # Convert mask to binary (1 = person, 0 = background)
        binary_mask = (mask > 0.5).astype(np.uint8)

        # Expand mask dimensions to match frame shape (for broadcasting)
        binary_mask = np.stack((binary_mask,) * 3, axis=-1)

        # Create a solid background (black) for the segmented frame
        background = np.zeros_like(frame, dtype=np.uint8)

        # Blend the original frame and the background using the mask
        output_frame = frame * binary_mask + background * (1 - binary_mask)

        # Show the result in real-time
        cv2.imshow('Real-Time Selfie Segmentation', output_frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


### For Blur or image background

In [3]:

# Initialize MediaPipe Selfie Segmentation
mp_selfie = mp.solutions.selfie_segmentation

# Start video capture
cap = cv2.VideoCapture(0)

# Load a custom background image (Optional: Uncomment to use an image)
background_image = cv2.imread("background.jpg")  # Replace with your image path
use_image_background = background_image is not None  # Set to True if image is loaded successfully

# Use 'with' statement for the segmentation model
with mp_selfie.SelfieSegmentation(model_selection=1) as model:  # Model 1 = Higher accuracy
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize background image to match the webcam frame
        if use_image_background:
            background_resized = cv2.resize(background_image, (frame.shape[1], frame.shape[0]))
        
        # Convert frame to RGB (MediaPipe requires RGB input)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with segmentation model
        res = model.process(frame_rgb)

        # Extract the segmentation mask
        mask = res.segmentation_mask

        # Convert mask to binary (1 = person, 0 = background)
        binary_mask = (mask > 0.5).astype(np.uint8)

        # Expand mask dimensions to match frame shape (for broadcasting)
        binary_mask = np.stack((binary_mask,) * 3, axis=-1)

        # **Option 1: Blurred Background**
        blurred_background = cv2.GaussianBlur(frame, (55, 55), 0)  # Apply Gaussian Blur

        # **Option 2: Image Background**
        background = background_resized if use_image_background else blurred_background  # Use image or blur

        # Blend the original frame and the background using the mask
        output_frame = frame * binary_mask + background * (1 - binary_mask)

        # Show the result in real-time
        cv2.imshow('Real-Time Selfie Segmentation', output_frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


### To choose among solid,blur or image Background
